In [1]:
# import numpy as np
# import matplotlib.pyplot as plt

# from Gillespie_evolution import EvolutionGillespie

In [2]:
# class written by ChatGPT
import numpy as np
import matplotlib.pyplot as plt

class EvolutionGillespie:
    """
    Class that simulates the evolution of a certain system, described by a set of 'chemical' reactions.

    Attributes:
    - number of different species present in the model (N)
    - list of reaction rates (coeff)
    - 3D array of reaction coefficients (reactions)
    - name (title): title of the system used for plots
    """

    def __init__(self, number_species, coeff, reactions, title):
        """
        Initialize the system.

        Parameters:
        - number_species (int): Number of species in the system.
        - coeff (list): List of reaction rates.
        - reactions (3D array): Reaction coefficients:
            * 1st dimension: 2 -> ingoing and outgoing coefficients.
            * 2nd dimension: Number of reaction equations present.
            * 3rd dimension: Number of species present (stoichiometric coefficients).
        - title (str): Title for plots.
        """
        self.N = number_species
        self.coeff = np.array(coeff)
        self.reactions = np.array(reactions)
        self.title = title

    def simulate(self, initial_state, t_max):
        """
        Simulate the system using Gillespie's algorithm.

        Parameters:
        - initial_state (list): Initial population of each species.
        - t_max (float): Maximum simulation time.

        Returns:
        - times (list): Time points.
        - populations (list of lists): Populations of each species at each time point.
        """
        # Initial setup
        state = np.array(initial_state)
        times = [0]
        populations = [state.tolist()]

        while times[-1] < t_max:
            # Calculate propensities for each reaction
            propensities = []
            for r in range(len(self.coeff)):
                rate = self.coeff[r]
                ingoing = self.reactions[0, r]
                propensity = rate
                for s in range(self.N):
                    if ingoing[s] > 0:
                        propensity *= np.math.comb(state[s], ingoing[s])  # n choose k
                propensities.append(propensity)

            total_propensity = sum(propensities)
            if total_propensity == 0:
                break  # No reactions can occur

            # Time until next reaction
            tau = np.random.exponential(1 / total_propensity)

            # Choose which reaction occurs
            reaction_index = np.random.choice(
                len(propensities), p=np.array(propensities) / total_propensity
            )

            # Update the state
            state += self.reactions[1, reaction_index] - self.reactions[0, reaction_index]
            times.append(times[-1] + tau)
            populations.append(state.tolist())

        return times, populations

    def plot_species(self, times, populations, species1, species2):
        """
        Plot the evolution of two species over time.

        Parameters:
        - times (list): Time points from the simulation.
        - populations (list of lists): Populations of each species at each time point.
        - species1 (int): Index of the first species to plot.
        - species2 (int): Index of the second species to plot.
        """
        populations = np.array(populations)
        plt.figure(figsize=(10, 6))
        plt.plot(times, populations[:, species1], label=f"Species {species1}")
        plt.plot(times, populations[:, species2], label=f"Species {species2}")
        plt.xlabel("Time")
        plt.ylabel("Population")
        plt.title(self.title)
        plt.legend()
        plt.grid()
        plt.show()


In [6]:
# Example: Simple Lotka-Volterra
number_species = 2
coeff = [0.1, 0.05, 0.2]  # Reaction rates: birth and death
reactions = np.array([
    # Ingoing
    [[2, 0], [3, 0]],  # Birth: nothing -> A; Death: A -> nothing
    # Outgoing
    [[0, 1], [0, 0]],
    # predator model
    [[1, 1], [0, 1]]
])
title = "Simple Birth-Death Process"

# Create the system
system = EvolutionGillespie(number_species, coeff, reactions, title)

# Simulate
initial_state = [10, 5]  # Start with 10 individuals of species 0
t_max = 500
times, populations = system.simulate(initial_state, t_max)

# Plot the evolution of species 0 and 1
system.plot_species(times, populations, 0, 1)

IndexError: index 2 is out of bounds for axis 1 with size 2